In [1]:
import warnings
warnings.filterwarnings('ignore')

# Markdown
* a document formatting approach that makes writing documentation (or anything) pretty easy
* can do markdown outside of jupyter
    * filename `somefile.md`
* you can make code cells

```python
def example_function(x):
    return f'I am {x}'
```



# Machine Learning Issues

![xkcd_curve_fitting.png](images/xkcd_curve_fitting.png)

# Let's consider a scatterplot
* at differing levels of complexity
    * complexity can refer to number of features, width of a matrix, etc
    * complexity here just means order of a polynomial
    
$$
f(x) = ax^4 + bx^3 + cx^2 + dx + C
$$

In [3]:
import numpy as np
import matplotlib.pyplot as plt

def plot_scatter_fit(num_samps=50, fit_complexity=1, degree=1):
    N = num_samps
    
    np.random.seed(seed=23)
    
    # gen rand x vals
    x = np.random.rand(N)*20
    
    
    # define polynomial
    if degree == 1:
        y = 5*x + 3
    elif degree == 2:
        y = 3*x**2 + 5*x + 3
    elif degree == 3:
        y = -0.2*x**3 + 3*x**2 + 5*x + 3
    elif degree == 4:
        y = 0.006*x**4 + -0.2*x**3 + 3*x**2 + 5*x + 3


    # help us draw a fit line
    fit = np.polyfit(x, y, 1)
    fit_fn = np.poly1d(fit)
    
    
    # get scatter range
    


plot_scatter_fit(num_samps=50, fit_complexity=1, degree=1)

[1.03459577e+01 1.89392521e+01 1.53091952e+01 5.64791688e+00
 4.42090727e+00 1.37244417e+01 3.34278406e+00 7.84884933e+00
 1.23610469e+01 8.23860190e+00 4.92976240e-02 1.76806436e+01
 1.76989508e+01 6.00819379e+00 1.17916373e+01 1.95685383e+01
 1.69018764e+01 1.30150878e+00 5.89488929e+00 5.75868882e+00
 1.64493268e+01 1.25236608e+01 2.20955428e+00 1.05762234e-02
 1.88433247e+01 2.83001515e+00 8.43193052e+00 6.92978880e+00
 1.73957017e+01 8.57203625e+00 1.65750297e+01 1.43570368e+01
 2.38453388e+00 1.19276835e+01 2.59512595e+00 1.55068183e+00
 1.66241051e+01 9.28771230e+00 3.24024958e+00 1.09595058e+01
 1.17697164e+01 1.54722634e+01 1.31169092e+01 1.11541352e+01
 3.56494533e+00 4.81167062e+00 1.01210926e+01 7.93491398e+00
 9.66110371e+00 1.91147968e+01]
